CSP.01
	Variablen V:{ # Eine Variable für jede Kombination von Eigenschaft und Hausnummer
		Hausfarbe(1), Nationalität(1), Haustier(1), Getränk(1), Zigarettenmarke(1),
		Hausfarbe(2), [..]	
	}
	Domänen D:{ # X ist Platzhalter für die Hausnummern 1-5
		Hausfarbe(x) : {rot, grün, weiß, gelb, blau}
		Nationalität(x) : {England, Spanien, Ukraine, Norwegen, Japan}
		Haustier(x) : {Hund, Schnecke, Fuchs, Pferd, Zebra}
		Getränk(x) : {Kaffee, Tee, Milch, Orangensaft, Wasser}
		Zigarettenmarke(x) : {Old-Gold, Kools, Chesterfield, Lucky-Strike, Parliaments}
	}
	Constraints: {
		Keine zwei Häuser dürfen dieselbe Hausfarbe, Nationalität, Haustiere, Getränk oder Zigarettenmarke haben.
		Nationalität(x) = England <-> Hausfarbe(x) = Rot
		Nationalität(x) = Spanien <-> Haustier(x) = Hund
		Getränk(x) = Kaffee <-> Hausfarbe(x) = Grün
		Nationalität(x) = Ukraine <-> Getränk(x) = Tee
		Hausfarbe(x) = Weiss <-> Hausfarbe(x+1) = Grün		# X ist Hausnummer von links nach rechts, x+1 also das Haus rechts von x
		Zigarettenmarke = Old-Gold <-> Haustier = Schnecken
		Zigarettenmarke = Kools <-> Hausfarbe = Gelb
		Getränk(3) = Milch
		Nationalität(1) = Norwegen
		Zigarettenmarke(x) = Chesterfield <-> Haustier(x+1) = Fuchs OR Haustier(x-1) = Fuchs
		Zigarettenmarke(x) = Kools <-> Haustier(x+1) = Pferd OR Haustier(x-1) = Pferd
		Zigarettenmarke(x) = Lucky-Strike <-> Getränk(x) = Orangensaft
		Nationalität(x) = Japan <-> Zigarettenmarke(x) = Parliaments
		Nationalität(x) = Norwegen <-> Hausfarbe(x+1) = Blau OR Hausfarbe(x-1) = Blau
	}

In [ ]:
class CSP:
    def init(self, variables, domains, neighbors, constraints):
        self.variables = variables
        self.domains = {v: set(domains[v]) for v in variables}
        self.neighbors = neighbors
        self.constraints = constraints

# Check if value assignment works with constraints
def consistent(csp, var, value, assignment):
    for other in csp.neighbors[var]:
        if other in assignment:
            if not csp.constraints(var, value, other, assignment[other]):
                return False
    return True

# When testing values for variables, check if other variables have a non-empty domain
def forward_check(csp, var, value, assignment):
    removed = []

    for neigh in csp.neighbors[var]:
        if neigh not in assignment:
            for v in list(csp.domains[neigh]):
                if not csp.constraints(var, value, neigh, v):
                    csp.domains[neigh].remove(v)
                    removed.append((neigh, v))

                # domain empty, tested value cannot be correct
                if len(csp.domains[neigh]) == 0:
                    # restore old values
                    for (n, val) in removed:
                        csp.domains[n].add(val)
                    return False, removed
    return True, removed

# Restore domains after backtrack
def restore_domains(csp, removed):
    for var, val in removed:
        csp.domains[var].add(val)

# Returns first variable without assigned value
def select_unassigned_variable(csp, assignment):
    for var in csp.variables:
        if var not in assignment:
            return var
    return None

# Backtracking search
def BT_Search(csp, assignment=None):
    if assignment is None:
        assignment = {}

    # CSP solved?
    if len(assignment) == len(csp.variables):
        return assignment

    # 2. Variable wählen
    var = select_unassigned_variable(csp, assignment)

    # 3. Werte ausprobieren
    for value in list(csp.domains[var]):
        if consistent(csp, var, value, assignment):

            assignment[var] = value

            # 4. Inference (Forward Checking)
            ok, removed = forward_check(csp, var, value, assignment)
            if ok:
                result = BT_Search(csp, assignment)
                if result is not None:
                    return result

            # Backtrack
            del assignment[var]
            restore_domains(csp, removed)

    return None

# CSP Definition

variables = {
}
domains = {
}
neighbors = {
}
def constraints(x, vx, y, vy):
    return True

Lösung: {'Farbe1': 'rot', 'Farbe2': 'grün', 'Farbe3': 'blau'}
